In [ ]:
!pip install ray[tune] tune-sklearn

In [ ]:
import torch
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler

data, y = torch.load('../input/titanic/data.pkl')
train_data, test_data = data[:len(y)], data[len(y):]

np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(train_data, y, train_size=.8, )

In [ ]:
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

std = StandardScaler()
model = AdaBoostClassifier()
pipe = Pipeline(
    [('std', std),
     ('model', model)]
)

cross_val_score(pipe, X_train, y_train, cv=10, n_jobs=-1, verbose=1).mean()


In [ ]:
# 超参数搜索
from ray import tune
from ray.tune.sklearn import TuneSearchCV

param = {
    'model__n_estimators': tune.randint(1, 500),
    'model__learning_rate': tune.loguniform(1e-2, 1),
    # 集成器参数，越多模型越复杂
    'model__max_depth': tune.randint(3, 100),
    # 树的深度，特征多模型多时限制深度
    'model__min_samples_split': tune.randint(2, len(train_data) + 1),
    # 样本大增大这个值，不大默认就行
    'model__min_samples_leaf': tune.randint(1, len(train_data) + 1),
    # 叶节点最小样本数，少于次值将被剪枝；
    # 如果样本量不大，不需要管这个值。如果样本量数量级非常大，则推荐增大这个值。
    'model__max_features': tune.randint(2, train_data.shape[1] + 1),
    # 划分时考虑的特征数量
    'model__max_leaf_nodes': tune.randint(2, train_data.shape[0] + 1)}
# 特征不多考虑，特征多可以考虑

tune_search = TuneSearchCV(
    pipe,
    param,
    search_optimization="bayesian",
    n_trials=-1,
    verbose=1,
    return_train_score=True,
    mode='max',
    time_budget_s=1800,
    error_score=np.nan,
    use_gpu=True,
    cv=10
)
tune_search.fit(train_data, y)
print(tune_search.best_params_, tune_search.best_score_)

In [ ]:
best_model = tune_search.best_estimator
best_model.score(train_data, y)

In [ ]:
import pandas as pd

best_model = tune_search.best_estimator
y_test = best_model.predict(test_data.values)
result = pd.DataFrame(y_test, index=test_data.index, columns=['Survived'], )
result.to_csv('rf.csv')
torch.save(tune_search.best_params_, 'rf.pkl')